In [2]:
from sklearn.datasets import fetch_openml
import numpy as np
import time
import torch
from scipy.optimize import linprog
from qpsolvers import solve_qp
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from torch.autograd import Function
import torch.nn as nn
from sklearn.model_selection import train_test_split
import sys
import StochasticGhost
import importlib

In [3]:
xd, yd = fetch_openml('mnist_784', version=1, return_X_y=True)
xd = (xd/255).astype('float32')

/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [22]:
class CustomNetwork(nn.Module):

    # For now the input data is passed as init parameters
    def __init__(self, layer_sizes, itrain, otrain_sevens, otrain_ones):
        super(CustomNetwork, self).__init__()

        # Create a list of linear layers based on layer_sizes
        self.itrain = itrain
        self.otrain_sevens = otrain_sevens
        self.otrain_ones = otrain_ones
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, x):
        #self.input = x
        for layer in self.layers[:-1]:
            x = torch.sigmoid(layer(x))
        x = self.layers[-1](x)
        return x
    
    def compute_loss(self, Y, Y_hat):
        """
        compute loss function
        """
        # L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
        L_sum = 0.5*torch.sum(torch.square(Y - Y_hat))

        m = Y.shape[0]
        L = (1./m) * L_sum

        return L
    
    def obj_fun(self, params, minibatch):
        # with torch.no_grad():
        x = self.itrain
        y_ones = self.otrain_ones
        model_parameters = list(self.parameters())
        samples = np.random.choice(len(y_ones), minibatch, replace=False)
        model_parameters[0].data = torch.Tensor(params[0])
        model_parameters[1].data = torch.Tensor(params[1])
        model_parameters[2].data = torch.Tensor(params[2])
        model_parameters[3].data = torch.Tensor(params[3])
        fval = self.compute_loss(self.forward(x[samples, :]), y_ones[samples])
        return fval.item()
    
    def obj_grad(self, params, minibatch):
        #f_grad = {}
        fgrad = []
        x = self.itrain
        y_ones = self.otrain_ones
        model_parameters = list(self.parameters())
        samples = np.random.choice(len(y_ones), minibatch, replace=False)
        #with torch.no_grad():
        model_parameters[0].data = torch.Tensor(params[0])
        model_parameters[1].data = torch.Tensor(params[1])
        model_parameters[2].data = torch.Tensor(params[2])
        model_parameters[3].data = torch.Tensor(params[3])
        for name, param in self.named_parameters():
            fgrad.append(torch.autograd.grad(
                self.compute_loss(self.forward(x[samples, :]), y_ones[samples]), param, retain_graph=True)[0].view(-1))
        
        fgrad.insert(4, torch.Tensor(params[4]).view(-1))
        fgrad.insert(5, torch.Tensor(params[5]).view(-1))

        fgrad = torch.cat(fgrad, dim=0)
        return fgrad
    
    def conf(self, params, minibatch):
        conf_val = None
        x = self.itrain
        y_sevens = self.otrain_sevens
        model_parameters = list(self.parameters())
        samples = np.random.choice(len(y_sevens), minibatch, replace=False)
        model_parameters[0].data = torch.Tensor(params[0])
        model_parameters[1].data = torch.Tensor(params[1])
        model_parameters[2].data = torch.Tensor(params[4])
        model_parameters[3].data = torch.Tensor(params[5])
        conf_val = self.forward(x[minibatch, :])
        conf_val = self.compute_loss(
            self.forward(x[samples, :]), y_sevens[samples])
        """
        TODO: Compute
        
        """
        return conf_val.item() - 0.015
    
    def conJ(self, params, minibatch):
        x = self.itrain
        y_sevens = self.otrain_sevens
        cgrad = []
        model_parameters = list(self.parameters())
        samples = np.random.choice(len(y_sevens), minibatch, replace=False)
        model_parameters[0].data = torch.Tensor(params[0])
        model_parameters[1].data = torch.Tensor(params[1])
        model_parameters[2].data = torch.Tensor(params[4])
        model_parameters[3].data = torch.Tensor(params[5])
        #with torch.no_grad():
        for name, param in self.named_parameters():
            #print(name)
            cgrad.append(torch.autograd.grad(
                self.compute_loss(self.forward(x[samples, :]), y_sevens[samples]), param, retain_graph=True)[0].view(-1))
        
        cgrad.insert(2, torch.Tensor(params[2]).view(-1))
        cgrad.insert(3, torch.Tensor(params[3]).view(-1))
        
        cgrad = torch.cat(cgrad, dim=0)
        """"
        TODO: Compute

        """
        return cgrad
    

In [5]:
yd = yd.astype(int)
ysevens = np.zeros((yd.shape[0],))
yones = np.zeros((yd.shape[0],))
ysevens[np.where(yd == 1)] = 1
yones[np.where(yd == 1)] = 1

In [6]:
def paramvals(maxiter, beta, rho, lamb, hess, tau, mbsz, numcon, geomp, stepdecay, gammazero, zeta, N, n):
    params = {
        'maxiter': maxiter,  # number of iterations performed
        'beta': beta,  # trust region size
        'rho': rho,  # trust region for feasibility subproblem
        'lamb': lamb,  # weight on the subfeasibility relaxation
        'hess': hess,  # method of computing the Hessian of the QP, options include 'diag' 'lbfgs' 'fisher' 'adamdiag' 'adagraddiag'
        'tau': tau,  # parameter for the hessian
        'mbsz': mbsz,  # the standard minibatch size, used for evaluating the progress of the objective and constraint
        'numcon': numcon,  # number of constraint functions
        'geomp': geomp,  # parameter for the geometric random variable defining the number of subproblem samples
        # strategy for step decrease, options include 'dimin' 'stepwise' 'slowdimin' 'constant'
        'stepdecay': stepdecay,
        'gammazero': gammazero,  # initial stepsize
        'zeta': zeta,  # parameter associated with the stepsize iteration
        'N': N,  # Train/val sample size
        'n': n  # Total number of parameters
    }
    return params

In [10]:
import types

In [11]:
def reload_package(root_module):
    package_name = root_module.__name__

    # get a reference to each loaded module
    loaded_package_modules = dict([
        (key, value) for key, value in sys.modules.items()
        if key.startswith(package_name) and isinstance(value, types.ModuleType)])

    # delete references to these loaded modules from sys.modules
    for key in loaded_package_modules:
        del sys.modules[key]

    # load each of the modules again;
    # make old modules share state with new modules
    for key in loaded_package_modules:
        print("Loading ", key)
        newmodule = __import__(key)
        oldmodule = loaded_package_modules[key]
        oldmodule.__dict__.clear()
        oldmodule.__dict__.update(newmodule.__dict__)

In [12]:
import StochasticGhost
reload_package(StochasticGhost)

Loading  StochasticGhost


In [23]:
maxiter = 1
layer_sizes = [70, 15, 1]
X_train = torch.tensor(xd.values)
Y_train_sevens = torch.tensor(ysevens)
Y_train_ones = torch.tensor(yones)
train_loss_list = []
val_loss_list = []
# X_train = torch.tensor()
net1 = CustomNetwork(
    layer_sizes, X_train[:, :70], Y_train_sevens, Y_train_ones)
nn_parameters = list(net1.parameters())
initw = [param.data for param in nn_parameters]
initw = initw + [nn_parameters[2].data, nn_parameters[3].data]
num_param = sum(p.numel() for p in net1.parameters()) + nn_parameters[2].numel() + nn_parameters[3].numel()
# net.train_and_update(xd, yd)
# output_t = net.forward(X_train[:, :70])
# output_v = net.forward(X_val[:, :70])
# obj = net.compute_loss(Y_train[:70], output_t)
# constraint = output_v-cons_bound
params = paramvals(maxiter=2, beta=10, rho=0.8, lamb=0.5, hess='diag', tau=1., mbsz=1,
                   numcon=1, geomp=0.7, stepdecay='dimin', gammazero=0.1, zeta=0.1, N=X_train.shape[0], n=num_param)
w, iterfs, itercs = StochasticGhost.StochasticGhost(
    net1.obj_fun, net1.obj_grad, [net1.conf], [net1.conJ], initw, params)

LOOKIE LOOKIE:::::::::  0.016842879115942413
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<class 'numpy.ndarray'>
<class 'numpy.float64'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.05308868 -0.03229936
 -0.04892499]
[0.         0.         0.         ... 0.12482169 0.13359711 0.25236037]
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<class 'numpy.ndarray'>
<class 'numpy.float64'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.05308868 -0.03229936
 -0.04892499]
[0.         0.         0.         ... 0.50398153 0.53936899 1.01894581]
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<class 'numpy.ndarray'>
<class 'numpy.float64'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.05308868 -0.03229936
 -0.04892499]
[ 0.          0.          0.         ... -0.48111743 -0.5158608
 -0.97515929

/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<class 'numpy.ndarray'>
<class 'float'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.05153339 -0.03309406
 -0.05164047]
[0.         0.         0.         ... 0.05519374 0.0581698  0.10767239]
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<class 'numpy.ndarray'>
<class 'float'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.05153339 -0.03309406
 -0.05164047]
[0.         0.         0.         ... 0.11038747 0.11633959 0.21534477]
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<class 'numpy.ndarray'>
<class 'float'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.05153339 -0.03309406
 -0.05164047]
[0.         0.         0.         ... 0.11038747 0.11633959 0.21534477]
<class 'numpy.ndarray'>
Cgrad Shape is:  1097
<cl